<a href="https://colab.research.google.com/github/nmcardoso/splus-model/blob/master/splus_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%tensorflow_version 1.x 
import random
import tensorflow as tf
import numpy as np
from keras import backend as K
import os
import glob
import multiprocessing as mp
import shutil
import pandas as pd

from keras import layers
from keras import models
from keras import optimizers
from keras import utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import VGG16

import matplotlib.pyplot as plt

from progressbar import progressbar

!pip3 install git+https://github.com/nmcardoso/fitsbook-python
import fitsbook as fb

TensorFlow 1.x selected.


Using TensorFlow backend.


  Cloning https://github.com/nmcardoso/fitsbook-python to /tmp/pip-req-build-be38z8e1
  Running command git clone -q https://github.com/nmcardoso/fitsbook-python /tmp/pip-req-build-be38z8e1
  Created wheel for fitsbook: filename=fitsbook-0.1-cp36-none-any.whl size=2181 sha256=94bb27182ca4e37eb8e209846cd11c7db95a886b9474b14645c51654b5f5df46
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ynbycdl/wheels/99/2a/a0/b37947be424acac83a373b37b38a2bfaa4557b049c19426655
Successfully built fitsbook


In [2]:
# Reprodutibilidade
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development

os.environ['PYTHONHASHSEED'] = '12'
np.random.seed(123)
random.seed(1234)
tf.set_random_seed(12345)

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, 
#                               inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

print('Sementes Plantadas!')

Sementes Plantadas!


In [0]:
!unzip /content/drive/'My Drive'/datasets/stamps_scaled.zip -d dataset

In [0]:
!tar -zxvf /content/drive/'My Drive'/datasets/dataset.tar.gz

In [0]:
!cp /content/drive/'My Drive'/datasets/stamps_scaled.csv /content/

In [0]:
class NpySequence(utils.Sequence):
  """Generate data for Keras"""
  def __init__(self, files, labels, classes, image_size=(256, 256), channels=3, batch_size=32):
    self.files = files
    self.labels = self.__to_categorical(labels, classes)
    self.image_size = image_size
    self.channels = channels
    self.batch_size = batch_size
    self.indexes = [i for i in range(len(files))]
    np.random.shuffle(self.indexes)
    # self.on_epoch_end()
  
  def __len__(self):
    """The number of batches per epoch"""
    return int(np.floor(len(self.files) / self.batch_size))
  
  def __getitem__(self, index):
    """Generate one batch of data"""
    partition = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    X, Y = self.__data_generation(partition)
    return X, Y
  
  def on_epoch_end(self):
    """Shuffle files list and remap labels"""
    np.random.shuffle(self.indexes)
    
  def __to_categorical(self, labels, classes):
    """Labels to numbers"""
    classes_dict = {c:i for i,c in enumerate(classes)}
    return [classes_dict[l] for l in labels]

  def __data_generation(self, partition):
    """Load npy file and associate with a label"""
    X = np.empty((self.batch_size, *self.image_size, self.channels))
    Y = np.empty((self.batch_size,), dtype=int)

    for i, index in enumerate(partition):
      X[i,] = np.load(self.files[index])
      Y[i] = self.labels[index]

    return X, Y

In [0]:
def preprocess():
  global train_generator, test_generator, val_generator
  df = pd.read_csv('/content/stamps_scaled.csv')
  E = df[df.CLASS == 'E']
  S = df[df.CLASS == 'S']

  train_df = pd.concat([E[0:3127], S[0:951]])
  train_X = [f'/content/dataset/{f}.npy' for f in train_df.ID.tolist()]
  train_Y = train_df.CLASS.tolist()

  val_df = pd.concat([E[3128:5004], S[952:1522]])
  val_X = [f'/content/dataset/{f}.npy' for f in val_df.ID.tolist()]
  val_Y = val_df.CLASS.tolist()

  test_df = pd.concat([E[5005:], S[1523:]])
  test_X = [f'/content/dataset/{f}.npy' for f in test_df.ID.tolist()]
  test_Y = test_df.CLASS.tolist()

  train_generator = NpySequence(train_X, train_Y, ['E', 'S'])
  val_generator = NpySequence(val_X, val_Y, ['E', 'S'])
  test_generator = NpySequence(test_X, test_Y, ['E', 'S'])

def compile_model(image_size):
  global model
  model = models.Sequential()
  model.name = 'splus_convolutional'
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size + (3,))))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(.5))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
                loss='binary_crossentropy',
                metrics=['acc'])

def fit(epochs=1):
  global model, history, train_generator, val_generator
  history = model.fit_generator(
      train_generator,
      steps_per_epoch=80,
      validation_steps=30,
      epochs=epochs,
      validation_data=val_generator,
      callbacks=[fb.callbacks.FitsbookCallback()],
      use_multiprocessing=True,
      workers=4
  )

In [0]:
preprocess()
compile_model((256, 256))
fit(300)

[Fitsbook]: Monitoring this training in real time https://natan.ninja/#/stats/32
Epoch 1/300
80/80 [==============================] - 148s 2s/step - loss: 0.5939 - acc: 0.7613 - val_loss: 0.5322 - val_acc: 0.7865
Epoch 2/300
80/80 [==============================] - 136s 2s/step - loss: 0.5690 - acc: 0.7574 - val_loss: 0.5500 - val_acc: 0.7615
Epoch 3/300
80/80 [==============================] - 153s 2s/step - loss: 0.5479 - acc: 0.7687 - val_loss: 0.5664 - val_acc: 0.7448
Epoch 4/300
80/80 [==============================] - 146s 2s/step - loss: 0.5380 - acc: 0.7742 - val_loss: 0.5351 - val_acc: 0.7708
Epoch 5/300
80/80 [==============================] - 139s 2s/step - loss: 0.5470 - acc: 0.7676 - val_loss: 0.5367 - val_acc: 0.7719
Epoch 6/300
80/80 [==============================] - 152s 2s/step - loss: 0.5519 - acc: 0.7586 - val_loss: 0.5488 - val_acc: 0.7583
Epoch 7/300
80/80 [==============================] - 143s 2s/step - loss: 0.5294 - acc: 0.7738 - val_loss: 0.5231 - val_acc: 0.